# About this Notebook

this notebook does the following...
- Combine the climate data on different csv files from `./ClimateData/`
- Inspet Missing Data & Replace Missing Data with Rolling Mean
- Combine YEAR and MONTH columns
- resample Monthly to Quarterly Climate Data
- Plot historical climate data
- Outputs `climateipynb_output.csv`|

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Combining the data from different stations

In [ ]:
# List of file paths
file_paths = ['./ClimateData/Baguio Monthly Data.csv', 
              './ClimateData/Baler Radar Monthly Data.csv', 
              './ClimateData/Cabanatuan Monthly Data.csv', 
              './ClimateData/Casiguran Monthly Data.csv', 
              './ClimateData/Clark Monthly Data.csv',
              './ClimateData/CLSU Monthly Data.csv', 
              './ClimateData/Cubi Point Monthly Data.csv', 
              './ClimateData/Dagupan Monthly Data.csv', 
              './ClimateData/Iba Monthly Data.csv', 
              './ClimateData/Sinait Monthly Data.csv',
              './ClimateData/Tuguegarao Monthly Data.csv']

# Initialize an empty DataFrame
df = pd.DataFrame()

# Read each CSV file and concatenate the data
for file_path in file_paths:
    data = pd.read_csv(file_path)
    df = pd.concat([df, data])

# df.to_csv('climatedata_combined.csv', index=False)
df.info()

# Data Cleaning

<strong> Climate Data Notes </strong>
<blockquote>
Notes: <br>
-999.0 indicate Missing Values <br>
RAINFALL UNIT: mm <br>
-1.0 indicate Trace Values (RAINFALL\<0.1mm) <br>
TMAX (Maximum Temperature, Unit: Deg C) <br>
TMIN (Minimum Temperature, Unit: Deg C) <br>
RH (Relative Humidity, Unit: %) <br>
Wind Speed, Unit: m/s <br>
Wind direction, Unit: degree relative to North indicating where the wind is blowing from <br>
</blockquote>

- replaced missing data (values -999) from each station with the rolling mean of the data (For all Features) 
- removing 'Trace Values' of RAINFALL

replacing the missing data was done first in each stations, before the data is combied, to avoid the rolling mean to be affected by other stations data.

In [ ]:
# Counting Missing Data
missing_data_count = df[df == -999].count()
print(missing_data_count)
# Counting Rainfall Trace Values
rainfall_count = (df['RAINFALL'] == -1.0).sum()
print('RAINFALL Trace Values: ', rainfall_count)

In [ ]:
df = pd.DataFrame()

# We are resolving the missing data by replacing it with the rolling mean of the data.
# We will also do this first in each station's data before concatenating them together.
for file_path in file_paths:
    data = pd.read_csv(file_path)

    # Replace -999 (Missing Data) with the rolling mean
    df_clean = data.replace(-999, data.mean().rolling(window=3).mean().round(1))
    # Convert WIND_SPEED to back to int
    df_clean['WIND_SPEED'] = df_clean['WIND_SPEED'].astype(int)

    df = pd.concat([df, df_clean])

# We also won't be using WIND_DIRECTION so we are removing that
df = df.drop('WIND_DIRECTION', axis=1)

df.to_csv('climatedabined.csv', index=False)
df.head()

In [ ]:
# Missing data in the dataset is labeled as -999, so we count that instead of NaN
missing_data_count = df[df == -999].count()
print(missing_data_count)

In [ ]:
df.info()

# Changing the table's structure

- Removing Wind_Direction
- Combining and Reformatting "YEAR" and "MONTH" Columns

# Plotting the Climate Data

In [7]:
plot_df = df.copy() # Copy the dataframe to to a new variable

In [ ]:
plot_df.head()

## By Quarter
by semester would be more fitting, but im just doing this so i have more choices for plots to show

In [ ]:
df.head()

In [10]:
# We have Year and Months columns in the dataset
# This codeblock reformats those two columns into a single DATE column
# and combines it by quarter for the plot

# Combine and reformat "YEAR" AND "MONTH" columns to "DATE" 
plot_df['DATE'] = pd.to_datetime(plot_df[['YEAR', 'MONTH']].assign(day=1)) + pd.offsets.MonthEnd(1)
plot_df = plot_df.drop(['YEAR', 'MONTH'], axis=1)

# Convert Monthly Data to Quarterly Data
plot_month = plot_df.resample('M', on='DATE').mean()
plot_quarter = plot_df.resample('Q', on='DATE').mean()
plot_semester = plot_df.resample('2Q', on='DATE').mean()

In [ ]:
plot_quarter.head()

In [ ]:
plot_quarter.plot(subplots=True, figsize=(10, 10))

In [ ]:
plot_semester.plot(subplots=True, figsize=(10, 10))

In [14]:
# Save combined data to a CSV file
plot_quarter.to_csv('climateipynb_output.csv')